In [1]:
import pandas as pd
import numpy as np
import string
import json

In [2]:
## Static Config
foreign_key = "Code"
names = ["Tap Number", "Owner/Representative Name"]

In [3]:
df = pd.read_csv('nep_wp_taps_data.csv')

In [4]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df.select_dtypes(include=numerics)
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num
df['Yield'] = 0

In [6]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [7]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [8]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

foreign_key_config = ""
name_config = []
for key in configs:
    if(configs[key].lower() == foreign_key.lower()):
        foreign_key_config = key
    if(configs[key].lower() in [x.lower() for x in names]):
        name_config.append(key)

configs['foreign_key'] = foreign_key_config
configs['names'] = name_config

In [9]:
### Replace Dataset Columns
df.columns = index

In [10]:
configs

{'A': 'Code',
 'B': 'WSS Name',
 'C': 'Tap Number',
 'D': 'Owner/Representative Name',
 'E': 'Contact Number',
 'F': 'Tap Type',
 'G': 'Metered Connection (Yes/No)',
 'H': 'Number of HH Served',
 'I': 'No. of HH (Brahmin/Chhetri)',
 'J': 'No. of HH (Janjati)',
 'K': 'No. of HH (Dalit)',
 'L': 'No. of HH (Minority)',
 'M': 'No. of HH having Toilet',
 'N': 'Male Population',
 'O': 'Female Population',
 'P': 'Tap Condition',
 'Q': 'Flow Condition',
 'R': 'Supply Hours',
 'S': 'Tap Water Quality',
 'T': 'Earthquake Effect',
 'U': 'Number of Complaints Logged this Year',
 'V': 'Numbers of Leakage observed in pipeline this year',
 'W': 'Yield',
 'foreign_key': 'A',
 'names': ['C', 'D']}

In [11]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 61005023,
  'B': 'Chamere Water Supply System',
  'C': 'T2',
  'D': 'Pirtha Singh Khadka',
  'E': -2147483648,
  'F': 'Community',
  'G': 'No',
  'H': 3,
  'I': 0,
  'J': 3,
  'K': 1,
  'L': 0,
  'M': 3,
  'N': 9,
  'O': 12,
  'P': 'Reconstruction',
  'Q': 'No water at all',
  'R': 24,
  'S': 'No turbidity',
  'T': 'None',
  'U': 0,
  'V': 0,
  'W': 0},
 {'A': 61005023,
  'B': 'Chamere Water Supply System',
  'C': 'T1',
  'D': 'Gopal Jung Bhandari',
  'E': -2147483648,
  'F': 'Community',
  'G': 'No',
  'H': 0,
  'I': 0,
  'J': 5,
  'K': 0,
  'L': 5,
  'M': 5,
  'N': 16,
  'O': 17,
  'P': 'Reconstruction',
  'Q': 'No water at all',
  'R': 24,
  'S': 'No turbidity',
  'T': 'None',
  'U': 0,
  'V': 0,
  'W': 0}]

In [12]:
### to json
df.to_json('../resources/js/data/nep_wp_taps_data.json', orient="records", indent=2)

with open('../resources/js/data/nep_wp_taps_config.json', 'w') as outfile:
    json.dump(configs, outfile)